In [1]:
from titrations.basics import *
from titrations.titrations2 import *
from titrations.examples import *

# Generic Titrations Demo
This is a low-level data structure example that could be the basis for a RESTful API.

This prototype consists of 3 modules:
1. `basics` consists of functionalities that are expected in an EHR. In a real-life application, an actual EHR API takes place of this.
2. `titrations` has the main elements of the applications, which will be covered below.
3. `examples` includes read-to-use examples of the concepts defined in `titrations`.

The basic premise of this prototype is that creating a medication titration algorithm should only require defining a few simple building blocks. There are three types of building blocks in this app: `DosingLadder`, `Rule`, and `Action`.

## Dosing Ladder

A `DosingLadder` is simply a group of ordered list of medication doses. Each `DosingLadder` contains all the possible doses within a certain medication class (eg, beta blockers). The following is an example of a `DosingLadder`.

In [2]:
beta_blocker_ladder = DosingLadder({
    "metoprolol succinate": [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    "carvedilol": [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    "bisoprolol": [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

Provided the information above, a `DosingLadder` can do a few useful things such as suggest the starting doses for this class of medication:

In [3]:
beta_blocker_ladder.lowest_steps

{'metoprolol succinate': metoprolol succinate 12.5 mg PO daily,
 'carvedilol': carvedilol 3.125 mg PO BID,
 'bisoprolol': bisoprolol 1.25 mg PO BID}

... or given a specific dose, suggest the next step up the dosing ladder:

In [4]:
beta_blocker_ladder.get_next_step_up(Medication(metoprolol_succinate, "25 mg", "PO", "daily"))

metoprolol succinate 50 mg PO daily

## Rule

A `Rule` is defined by the name of patient parameter (eg, heart rate), a value, and an operation. A `Rule` can then be used to evaluate patients and see if they meet or trigger the rule.

The following is an example of a rule that evaluates for systolic blood pressure less than 100:

In [5]:
hypotension = Rule('SBP', 'lt', 100)

`"lt"` refers to the boolean operation less than (`<`).

The operations recognized by `Rule` are `"eq"` (`==`), `"neq"` (`!=`), `"gt"` (`>`), `"gte"` (`>=`), `"lt"` (`<`), "`lte"` (`<=`), and `"in"` (`in`). 


Using this rule, we can evaluate a particular patient as follows. In this example, the patient triggers the rule:

In [6]:
p = Patient(SBP=90)
hypotension.evaluate(p).is_satisfied

True

The following patient does not trigger the rule:

In [7]:
p = Patient(SBP=110)
hypotension.evaluate(p).is_satisfied

False

A `Rule` can also be conditional, meaning that it is triggered only in the presence of a certain condition. The condition is defined as another `Rule` as well. For example:

In [8]:
no_pacemaker = Rule('has_pacemaker', 'eq', False)
bradycardia = ConditionalRule('HR' , 'lt', 60, condition=no_pacemaker)

In the following example, although the patient meets the `Rule`'s criteria, the rule is not triggered because the condition is not met.

In [9]:
p = Patient(HR=50, has_pacemaker=True)
bradycardia.evaluate(p).is_satisfied

False

## Action

An `Action` is a single recommendation that could be made by the titration algorithm, such as starting a medication or increasing the dose of a medication. An `Action` contains the logic for generating a text version of the recommendation, generating a button, and providing a hook which executes the action upon triggering the button.

The following is an example of an action:

In [10]:
class Start(Action):
    """
    Start a new medication.
    """
    def suggest(self):
        """
        Provide a text recommendation.
        """
        med_names = list(self.lowest_steps)
        return f"Start {', '.join([str(self.lowest_steps[med]) for med in med_names[:-1]])}, or {str(self.lowest_steps[med_names[-1]])}."

    def buttons(self):
        """
        Provide a list of buttons to display.
        """
        return [str(self.lowest_steps[med]) for med in self.lowest_steps]

    def perform(self, medication_name: str):
        """
        Perform the action.
        """
        assert medication_name in self.lowest_steps
        self.patient.medications.append(self.lowest_steps[medication_name])

    def __init__(self, patient: Patient, dosing_ladder: DosingLadder):
        super().__init__(patient, dosing_ladder, None)
        self.lowest_steps = self.dosing_ladder.lowest_steps

The `Action` takes a `Patient` and a `DosingLadder` as parameters. The following is an example of instantiating an `Action` using a `Patient` with no medications.

In [11]:
p = Patient(medications=[])
start = Start(p, beta_blocker_ladder)

The `Action` can now return a text recommendation:

In [12]:
start.suggest()

'Start metoprolol succinate 12.5 mg PO daily, carvedilol 3.125 mg PO BID, or bisoprolol 1.25 mg PO BID.'

It can also return a list of buttons (this is just a list of strings now, but I'm pretending these are buttons for the purposes of this illustration):

In [13]:
start.buttons()

['metoprolol succinate 12.5 mg PO daily',
 'carvedilol 3.125 mg PO BID',
 'bisoprolol 1.25 mg PO BID']

It can also perform the action once triggered.

In this example, the `Action` modifies the patient's medication list. In a real-world application, this could pend a prescription for the provider to sign. The value `"metoprolol succinate"` is passed back to the method upon clicking the appropriate button.

In [14]:
start.perform("metoprolol succinate")
p.medications

[metoprolol succinate 12.5 mg PO daily]

The `titrations` module comes with a few pre-defined `Actions`:
- `Start`
- `DoNotStart`
- `StepUp`
- `StepDown`
- `Continue`
- `Stop`
- `MarkMaxDose`
- `ReportReaction`

(Note: not all of these are fully implemented)

## Rule With Actions

A `RuleWithActions` is a `Rule` that, in addition to returning a value of `True` or `False`, returns a list of recommended actions depending on the evaluation result of the rule.

For example, the rule `hypotension` could return `StepDown` and `MarkMaxDose` as recommended actions when it is triggered.

Since there are some common patterns in how certain rules provide recommendations, the following generic `Rule(s)WithActions` are provided:

In [15]:
class ClassLimitingRule(RuleWithActions):
    default_actions_when_satisfied = [Stop, ReportReaction]

class TitrationLimitingRule(RuleWithActions):
    default_actions_when_satisfied = [Continue, StepDown, MarkMaxDose]

Using these tools, the `hypotension` rule can now be defined as:

In [16]:
hypotension = TitrationLimitingRule('SBP', 'lt', 100)

In [17]:
p = Patient(SBP=90)
eval_result = hypotension.evaluate(p)
eval_result.is_satisfied

True

In [18]:
eval_result.recommended_actions

[titrations.titrations2.Continue,
 titrations.titrations2.StepDown,
 titrations.titrations2.MarkMaxDose]

## Titrator

A `Titrator` combines all the concepts above and is the workhorse of the algorithm. Different titrators are defined as subclasses of `Titrator`. An example is below:

In [19]:
class BetaBlockerHFrEFTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    default_titration_target = MaxTolerated
    default_rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

In [ ]:
normal_blood_pressure = Rule('SBP', 'lt', 120)

class BetaBlockerHypertensionTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    default_titration_target = normal_blood_pressure
    default_rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

### Example 1

The following is an example of instantiating and running the titrator on a particular patient. In this case, the patient is not on any medications, and has no contraindications to start a beta blocker. After evaluation, the attribute `can_advance` indicates whether the next step in titrating the medication can be taken, whether that be initiating or uptitrating a medication.

In [23]:
patient1 = Patient(SBP=110, HR=70, medications=[], has_pacemaker=False, decompensated=False, av_block=False, symptomatic=False)

titrator1 = BetaBlockerHFrEFTitrator(patient1)
titrator1.evaluate()

titrator1.can_advance

True

The attribute `recommended_actions` indicates the `Action`s that resulted from the evaluation 
of each of the rules within the titrator. In this case, it's recommending a `Start` action.

In [24]:
titrator1.recommended_actions

The same properties and methods of `Action`s are accessible, similar to what's described above. Here we ask for a text recommendation.

In [25]:
titrator1.recommended_actions[0].suggest()

'Start metoprolol succinate 12.5 mg PO daily, carvedilol 3.125 mg PO BID, or bisoprolol 1.25 mg PO BID.'

### Example 2

In this example, the patient has low systolic blood pressure, and is decompensated. The titrator indicates that advancing is not recommended, as indicated by the `can_advance` attribute.

In [34]:
patient2 = Patient(SBP=70, HR=70,
                   medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")],
                   has_pacemaker=False, decompensated=True, av_block=False, symptomatic=False)

titrator2 = BetaBlockerHFrEFTitrator(patient2)
titrator2.evaluate()

titrator2.can_advance

False

It also explains why it cannot advance by providing a list of rules that have been triggered, using the attribute `satisfied_rules`:

In [27]:
titrator2.satisfied_rules

[SBP lt 100, decompensated eq True]

It subsequently provides a different list of recommended actions. These are provided by the two rules that have been triggered.

In [28]:
titrator2.recommended_actions

Similar to above, these actions have the full capabilities of `Action`s:

In [35]:
titrator2.recommended_actions[1].suggest()

'Decrease metoprolol succinate to metoprolol succinate 12.5 mg PO daily.'

## Full Example

Putting everything together, the language described above can be used to express a full titration algorithm. The HFrEF beta blocker titration algorithm can be fully expressed using the following code:

In [ ]:
# Dosing ladder

beta_blocker_ladder = DosingLadder({
    metoprolol_succinate.name: [
        Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
        Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
    ],
    carvedilol.name: [
        Medication(carvedilol, "3.125 mg", "PO", "BID"),
        Medication(carvedilol, "6.25 mg", "PO", "BID"),
        Medication(carvedilol, "12.5 mg", "PO", "BID"),
        Medication(carvedilol, "25 mg", "PO", "BID"),
    ],
    bisoprolol.name: [
        Medication(bisoprolol, "1.25 mg", "PO", "BID"),
        Medication(bisoprolol, "2.5 mg", "PO", "BID"),
        Medication(bisoprolol, "5 mg", "PO", "BID"),
        Medication(bisoprolol, "10 mg", "PO", "BID"),
    ]
})

# Rules

no_pacemaker = Rule('has_pacemaker', 'eq', False)

hypotension = TitrationLimitingRule('SBP', 'lt', 100)
bradycardia = ConditionTitrationLimitingRule('HR' , 'lt', 60, condition=no_pacemaker)
decompensation = TitrationLimitingRule('decompensated', 'eq', True)
symptoms = TitrationLimitingRule('symptomatic', 'eq', True)
av_block = TitrationLimitingRule('av_block', 'eq', True)

# Titrator

class BetaBlockerHFrEFTitrator(Titrator):
    dosing_ladder = beta_blocker_ladder
    default_titration_target = MaxTolerated
    default_rules = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
        av_block,
    ]

# Assumptions and Limitations

# Further direction